In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("rsmp/reestr_group_A/data_product.csv", dtype=str)

In [3]:
data.head(3)

,id,tin,reg_number,kind,category,first_name,last_name,patronymic,org_name,org_short_name,...,region,area,settlement,settlement_type,oktmo,lat,lon,address_raw,start_date,end_date
0,0,0105035949,1020100709111,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "" ЛЮЦ...",ЛЮЦЕРНА АГРИКОЛ ООО,...,Республика Адыгея,Городской округ Майкоп,Западный,п,79701000121.0,44.6316666666667,40.0719444444444,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / З...,2016-08-10,2023-11-10
1,1,0105075540,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ДАГМАЙ""","ООО ""ДАГМАЙ""",...,Республика Адыгея,NaN,Майкоп,г,79701000001.0,44.6098268,40.1006606,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / /,2017-08-10,2020-07-10
2,2,0218000256,1020201626686,1,1,NaN,NaN,NaN,СЕЛЬСКОХОЗЯЙСТВЕННЫЙ ПРОИЗВОДСТВЕННЫЙ КООПЕРАТ...,СПК ИМЕНИ САЛАВАТА,...,Республика Башкортостан,Бурзянский,Кутаново,д,80619416106.0,52.9833333333333,57.0327777777778,БАШКОРТОСТАН / РЕСПУБЛИКА / БУРЗЯНСКИЙ / РАЙОН...,2016-08-10,2023-11-10


In [4]:
data["start_date"] = pd.to_datetime(data["start_date"])
data["end_date"] = pd.to_datetime(data["end_date"])

In [5]:
start_year = data["start_date"].dt.year.min()
end_year = data["end_date"].dt.year.max()

In [6]:
panel_elems = []
for year in range(start_year, end_year + 1):
    start_dt = pd.Timestamp(f"{year}-01-01")
    end_dt = pd.Timestamp(f"{year}-12-31")
    yearly_data = data.sort_values("start_date").loc[~((data["start_date"] > end_dt) | (data["end_date"] < start_dt))].copy()
    group_sizes = yearly_data.groupby("tin", as_index=False).size()
    yearly_data.drop_duplicates(subset=["tin"], keep="last", inplace=True)
    yearly_data["year"] = year    
    yearly_data = yearly_data.merge(group_sizes, how="left", on="tin")
    yearly_data["confidence"] = 1 / yearly_data["size"]
    yearly_data.drop(columns=["start_date", "end_date", "size"], inplace=True)
    panel_elems.append(yearly_data)
panel = pd.concat(panel_elems)
panel.shape

(1806561, 21)

In [7]:
panel.head(3)

,id,tin,reg_number,kind,category,first_name,last_name,patronymic,org_name,org_short_name,...,region,area,settlement,settlement_type,oktmo,lat,lon,address_raw,year,confidence
0,0,0105035949,1020100709111,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ "" ЛЮЦ...",ЛЮЦЕРНА АГРИКОЛ ООО,...,Республика Адыгея,Городской округ Майкоп,Западный,п,79701000121.0,44.6316666666667,40.0719444444444,АДЫГЕЯ / РЕСПУБЛИКА / / / МАЙКОП / ГОРОД / З...,2016,1.0
1,513095,3306014623,NaN,1,1,NaN,NaN,NaN,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""АГРО...","ООО ""АГРОФИРМА МИР""",...,Владимирская область,Кольчугинский район,Кольчугино,г,17640101001.0,56.3327254,39.391336,ВЛАДИМИРСКАЯ / ОБЛАСТЬ / КОЛЬЧУГИНСКИЙ / РАЙОН...,2016,1.0
2,240850,507702785290,NaN,2,1,ГЕОРГИЙ,ТАБОЛОВ,ВИКТОРОВИЧ,NaN,NaN,...,Москва,NaN,NaN,г,45000000.0,55.7540471,37.620405,МОСКВА / ГОРОД / / / / / /,2016,1.0


In [8]:
panel.to_csv("rsmp/reestr_group_A/panel.csv", index=False)